# SQL to Spark SQL Translator for Azure Databricks

This notebook assists in translating T-SQL stored procedures to Spark SQL for use in Azure Databricks. It validates input, identifies dependencies, and generates Spark SQL code.


## Step 0: Prerequisites
- Ensure you have access to an Azure OpenAI subscription with a gpt-4o model provisioned. Increase the token per minute limit (TPM) to at least 130K. 
- Duplicate the .env.sample file and name it .env before populating it.
- Install necessary libraries: (you can also run `pip install -r requirements.txt` in the terminal instead of this cell)


In [ ]:
# Install required packages
%pip install pandas sqlglot openai networkx matplotlib python-dotenv tiktoken

- Load the dependencies and define common functions

In [ ]:
# Load libraries and environment variables
import os
import pandas
import networkx
import matplotlib
from IPython.display import Markdown, display
from dotenv import load_dotenv

load_dotenv()

# Define GPT-4o query function
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("AZURE_OPENAI_PREVIEW_API_VERSION"),
)

temperature = float(os.getenv("AZURE_OPENAI_TEMPERATURE"))
max_tokens = os.getenv("AZURE_OPENAI_MAX_TOKENS")

def gpt_4o_analysis(system_message, user_query):
    response = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": f"The user's query is: {user_query}"}
        ],
        temperature=temperature
        # max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

print("Loaded libraries, environment variables, and defined GPT-4o query function.")


## Step 1: Provide T-SQL Stored Procedure and Select Run Options

Please paste your T-SQL stored procedure in the input cell below, or set read_from_file to true and populate bulk.sql:

In [ ]:
# Set this variable to true to read from the file:
read_from_file = True

if read_from_file:
	# Import a file containing SQL and read it into tsql_procedure; each statement must be separated by a semicolon
	with open('bulk.sql', 'r') as file:
		bulk_sql = file.read()

	# Split ddl.sql into a list of strings by semicolons
	bulk_sql_list = bulk_sql.split(';')

# User input for T-SQL stored procedure
tsql_procedure = """



/** Your T-SQL code goes here **/




"""
print("Successfully loaded T-SQL stored procedure.")

## Step 2: Input Validation and Preprocessing

The tool will now validate and preprocess your T-SQL syntax to mitigate syntax-induced errors that occur before conversion.

In [ ]:
if not read_from_file:
    # Evaluate the T-SQL syntax correctness
    system_message = "You are an AI assistant that helps validate T-SQL syntax. Identify only extremely critical errors that will prevent the SQL from executing. The dialect is T-SQL."
    user_query = tsql_procedure # "CREATE PROS dbo.TestTable (ID int, Name varchar(50));" # Wrong example to test
    result = gpt_4o_analysis(system_message, "Evaluate the following T-SQL query. Apply all rules of the T-SQL dialect. Do not reprint the entire corrected script. Show only extremely major errors. Miscapitalized or abbreviated but correct keywords are not errors. Be as concise as possible. If no errors are detected, simply state so. \n\nSQL Query: \n\n" + user_query)
    display(Markdown(result))

else:
    for query in bulk_sql_list:
        # Evaluate the T-SQL syntax correctness
        system_message = "You are an AI assistant that helps validate T-SQL syntax. Identify only extremely critical errors that will prevent the SQL from executing. The dialect is T-SQL."
        user_query = query
        result = gpt_4o_analysis(system_message, "Evaluate the following T-SQL query. Apply all rules of the T-SQL dialect. Do not reprint the entire corrected script. Show only extremely major errors. Miscapitalized or abbreviated but correct keywords are not errors. Be as concise as possible. If no errors are detected, simply state so. \n\nSQL Query: \n\n" + user_query)
        # Display the result and the original query, formatted for readability

        display(Markdown(f"### Original Query: \n\n```sql\n{user_query}\n```\n\n"))
        display(Markdown(result))

        print("\n\n\n")

## Step 3: Dependency Analysis

The tool will now analyze dependencies such as tables, views, and other database objects.

In [ ]:
# Dependency analysis function using GPT-4o
def analyze_dependencies(tsql):
    system_message = "You are an AI assistant that helps identify dependencies in T-SQL scripts. Extract and list all tables, views, and other database objects that the script depends on."
    user_query = tsql
    result = gpt_4o_analysis(system_message, "Analyze the following T-SQL query and list all dependencies (tables, views, and other database objects). Be as concise as possible. Return each dependency in a newline and provide no commentary text. \n\nSQL Query: \n\n" + user_query)
    
    print(result)

    # Parse the result to extract dependencies
    dependencies = []
    # for line in result.split('\n'):
    #     dependencies.append(line.strip())

    dependencies = list(filter(None, result.split("\n")))
    
    return dependencies

if not read_from_file:
    dependencies = analyze_dependencies(tsql_procedure)
    print("Dependencies identified:", dependencies)

else:
    for query in bulk_sql_list:
        dependencies = analyze_dependencies(query)
        # Display the result and the original query, formatted for readability
        display(Markdown(f"### Original Query: \n\n```sql\n{query}\n```\n\n"))
        print("Dependencies identified:", dependencies)

        print("\n\n\n")

## Step 4: Token and Length Limit Check

The total **context window** (combined input and output tokens) for the Azure OpenAI GPT-4o model must not exceed 128K. We use tiktoken to check the total length of the input and multiply it by two for a rough estimate of whether the script is too long for this translator to handle.

In [ ]:
import tiktoken

# Initialize tiktoken encoding
encoding = tiktoken.encoding_for_model("gpt-4o")

if not read_from_file:
    # Encode the T-SQL procedure to get the token count
    tokens = encoding.encode(tsql_procedure)
    input_token_count = len(tokens)

    # Estimate the total context window needed (input tokens * 2)
    estimated_total_context_window = input_token_count * 2

    # Define the maximum context window for Azure OpenAI GPT-4o model
    max_context_window = 128000

    # Check if the estimated context window is within the allowable limit
    if estimated_total_context_window <= max_context_window:
        print("The T-SQL procedure is within the allowable context window for translation.")
    else:
        print("The T-SQL procedure exceeds the allowable context window for translation.")
        print(f"Estimated total context window: {estimated_total_context_window} tokens")

    print(f"Input token count: {input_token_count}")

else:
    for query in bulk_sql_list:
        # Encode the T-SQL query to get the token count
        tokens = encoding.encode(query)
        input_token_count = len(tokens)

        # Estimate the total context window needed (input tokens * 2)
        estimated_total_context_window = input_token_count * 2

        # Define the maximum context window for Azure OpenAI GPT-4o model
        max_context_window = 128000

        # Display the result and the original query, formatted for readability
        display(Markdown(f"### Original Query: \n\n```sql\n{query}\n```\n\n"))

        # Check if the estimated context window is within the allowable limit
        if estimated_total_context_window <= max_context_window:
            print("The T-SQL query is within the allowable context window for translation.")
        else:
            print("The T-SQL query exceeds the allowable context window for translation.")
            print(f"Estimated total context window: {estimated_total_context_window} tokens")

        print(f"Input token count: {input_token_count}")

        print("\n\n\n")


## Step 5/6: Translation to Spark SQL

The tool will now translate the T-SQL stored procedure to Spark SQL.

In [ ]:
import datetime
import os

# Translation to Spark SQL using GPT-4o
def translate_to_spark_sql(tsql):
    system_message = "You are an AI assistant that helps translate T-SQL to Spark SQL. The dialect is Spark SQL."
    user_query = tsql + " ---------- \n\n You must ensure that all referenced schemas do not contain square brackets, and are true to their source of stg_dbo or NO PREFIX if dbo."
    result = gpt_4o_analysis(system_message, "Translate the following T-SQL query to Spark SQL that will be executed in an Azure Databricks environment. Be as precise and accurate as possible. Provide no additional commentary, returning just the translated query. Remember, stored procedures don't exist in Databricks. \n\nT-SQL Query: \n\n" + user_query)
    return result

if not read_from_file:
    spark_sql_procedure = translate_to_spark_sql(tsql_procedure)
    print("Translated Spark SQL:")
    display(Markdown(spark_sql_procedure))

else:
    for query in bulk_sql_list:
        spark_sql_query = translate_to_spark_sql(query)
        # Display the result and the original query, formatted for readability
        display(Markdown(f"### Original Query: \n\n```sql\n{query}\n```\n\n"))
        print("Translated Spark SQL:")
        display(Markdown(spark_sql_query))

        # Write same format to a new .txt file with custom timestamp in the filename
        
        # Create the output folder if it doesn't exist
        output_folder = "output_spark_sql"
        os.makedirs(output_folder, exist_ok=True)
        
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        file_name = f"spark_sql_{timestamp}.txt"
        file_path = os.path.join(output_folder, file_name)
        
        with open(file_path, "w") as text_file:
            text_file.write("### ORIGINAL QUERY \n\n\n""" + query + "\n\n" + "### TRANSLATED QUERY \n\n\n" + spark_sql_query)

        print(f"File '{file_name}' created in the '{output_folder}' folder.")


        print('\n\n\n')



<br><br><br>

## Step 5/7: Translation to PySpark

The tool will now translate the T-SQL stored procedure to PySpark.

In [ ]:
import datetime
import os

# Translation to PySpark using GPT-4o
def translate_to_pyspark(tsql):
    system_message = "You are an AI assistant that helps translate T-SQL to PySpark."
### NOTE: IF YOU NEED TO REGENERATE WITH SPECIAL REQUESTS (i.e. bug fixes), SPECIFY CHANGES HERE:
    user_query = """
     Assume that dataframes have NOT yet been instantiated for mentioned tables.
     For all tables preceded by "dbo.", assume they are in the default database and remove the prefix.
     This is because the tables have been migrated to the default database in Azure Databricks.
     You must ensure that all referenced schemas do not contain square brackets, and are true to their source of stg_dbo or NO PREFIX if dbo.
     Note that case sensitivity matters for PySpark table references.
    """

    result = gpt_4o_analysis(system_message, "Your goal is to translate  the following T-SQL query to PySpark that will be executed in Azure Databricks environment. You don't need to configure or initialize a Spark context or Spark session, as these are managed for you by Azure Databricks. Be as precise and accurate as possible. Provide no additional commentary, returning just the translated query. " + user_query + "\n\nT-SQL Query: \n\n" + tsql)
    return result

if not read_from_file:
    pyspark_procedure = translate_to_pyspark(tsql_procedure)
    print("Translated PySpark:")
    display(Markdown(pyspark_procedure))

else:
    for query in bulk_sql_list:
        pyspark_procedure = translate_to_pyspark(query)
        # Display the result and the original query, formatted for readability
        display(Markdown(f"### Original Query: \n\n```sql\n{query}\n```\n\n"))
        print("Translated PySpark:")
        display(Markdown(pyspark_procedure))

        # Write same format to a new .txt file with custom timestamp in the filename
        
        # Create the output folder if it doesn't exist
        output_folder = "output_pyspark"
        os.makedirs(output_folder, exist_ok=True)
        
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        file_name = f"pyspark_{timestamp}.txt"
        file_path = os.path.join(output_folder, file_name)
        
        with open(file_path, "w") as text_file:
            text_file.write("### ORIGINAL QUERY \n\n\n""" + query + "\n\n" + "### TRANSLATED PYSPARK \n\n\n" + pyspark_procedure)

        print(f"File '{file_name}' created in the '{output_folder}' folder.")

        print("\n\n\n")
